##### Copyright 2019 The TensorFlow Authors.

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
PATH = 'tf-weights/weights'
QUANTIZED_MODEL_PATH = "tf-weights/converted_quant_model.tflite"

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(10, use_bias=False)
])

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.7314889 , -0.50046164,  0.35221618, -0.36701477,  0.26105803,
        -0.4982737 , -0.425567  ,  0.7033225 ,  0.04746384, -0.24647425]],
      dtype=float32)

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [7]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss: 

In [8]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 2s 919us/step - loss: 0.4732 - accuracy: 0.8756
Epoch 2/5
1875/1875 [==============================] - 2s 988us/step - loss: 0.3078 - accuracy: 0.9145
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2878 - accuracy: 0.9191
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2782 - accuracy: 0.9224
Epoch 5/5
1875/1875 [==============================] - 2s 992us/step - loss: 0.2721 - accuracy: 0.9243


In [9]:
model.save_weights(PATH)

The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [10]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.2718 - accuracy: 0.9245


[0.2717747092247009, 0.9244999885559082]

In [11]:
def rep_data_gen():
    for x in x_train:
        yield [x.astype(np.float32)]

In [12]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
converter.representative_dataset = rep_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model = converter.convert()
open(QUANTIZED_MODEL_PATH, "wb").write(tflite_quant_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:\Users\skarn\AppData\Local\Temp\tmpj67tvg2p\assets


9296

In [13]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=QUANTIZED_MODEL_PATH)
interpreter.allocate_tensors()
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [16]:
output_details

[{'name': 'Identity',
  'index': 4,
  'shape': array([ 1, 10]),
  'shape_signature': array([-1, 10]),
  'dtype': numpy.int8,
  'quantization': (0.24100109934806824, 58),
  'quantization_parameters': {'scales': array([0.2410011], dtype=float32),
   'zero_points': array([58]),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [15]:
tensor.

SyntaxError: invalid syntax (Temp/ipykernel_19320/2424401444.py, line 1)